# samsum 数据导入

In [1]:
import torch
import datasets
from transformers import pipeline

dataset_samsum = datasets.load_from_disk('../../../resources/dataset/hf_samsum/')
# pipe = pipeline("summarization", model="../../../resources/embedding/google_pegasus-cnn-dm") 

/Users/apollo/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: Directory ../../../resources/dataset/hf_samsum/ not found

In [3]:
from datasets import load_metric 
import pandas as pd 
bleu_metric = load_metric("sacrebleu")
rouge_metric = load_metric("rouge") # rouge_score==0.0.4 work well

/tmp/ipykernel_125004/2994031358.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")
Using the latest cached version of the module from /home/users/sunhongchao/.cache/huggingface/modules/datasets_modules/metrics/sacrebleu/31e1673407d8789b8f5ddfd979948f6a1de0a6d691426d55fa74a35ffb0c1bdf (last modified on Sat Oct 29 23:06:04 2022) since it couldn't be found locally at sacrebleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/users/sunhongchao/.cache/huggingface/modules/datasets_modules/metrics/rouge/0ffdb60f436bdb8884d5e4d608d53dbe108e82dac4f494a66f80ef3f647c104f (last modified on Sat Oct 29 23:10:57 2022) since it couldn't be found locally at rouge, or remotely on the Hugging Face Hub.


In [4]:
from tqdm import tqdm
device = "cuda:1" if torch.cuda.is_available() else "cpu" 
print(device)
def chunks(list_of_elements, batch_size): 
	"""Yield successive batch-sized chunks from list_of_elements.""" 
	for i in range(0, len(list_of_elements), batch_size): 
	    yield list_of_elements[i : i + batch_size] 
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="dialogue", column_summary="summary"): 
	article_batches = list(chunks(dataset[column_text], batch_size)) 
	target_batches = list(chunks(dataset[column_summary], batch_size)) 
	for article_batch, target_batch in tqdm( zip(article_batches, target_batches), total=len(article_batches)): 
		inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 
		summaries = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device), length_penalty=0.8, num_beams=8, max_length=128) 
		decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries] 
		decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries] 
		metric.add_batch(predictions=decoded_summaries, references=target_batch) 
	score = metric.compute() 
	return score

cuda:1


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer 

model_ckpt = "../../../resources/embedding/google_pegasus-cnn-dm" 
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device) 

In [6]:
def convert_examples_to_features(example_batch): 
	input_encodings = tokenizer(example_batch["dialogue"], max_length=1024, truncation=True) 
	# with tokenizer.as_target_tokenizer(): 
	target_encodings = tokenizer(example_batch["summary"], max_length=128, truncation=True) 
	return {"input_ids": input_encodings["input_ids"], "attention_mask": input_encodings["attention_mask"], "labels": target_encodings["input_ids"]} 
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True) 
columns = ["input_ids", "labels", "attention_mask"] 
dataset_samsum_pt.set_format(type="torch", columns=columns)

Loading cached processed dataset at ../../../resources/dataset/hf_samsum/train/cache-c5296302575c0f7a.arrow
  0%|          | 0/1 [00:00<?, ?ba/s]
Loading cached processed dataset at ../../../resources/dataset/hf_samsum/validation/cache-7bc2f4bf3670c069.arrow


In [16]:
import os
os.environ["WANDB_API_KEY"] = "12fe58dbecee4ddfa653d6ac2ad2f09612ddecd0"
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_DISABLED"] = "true"
# import wandb
# wandb.init()

In [17]:
from transformers import DataCollatorForSeq2Seq 
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) 
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"] 
from transformers import TrainingArguments, Trainer 
training_args = TrainingArguments( output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500, per_device_train_batch_size=1, per_device_eval_batch_size=1, weight_decay=0.01, logging_steps=10, push_to_hub=False,
evaluation_strategy='steps', eval_steps=500, save_steps=1e6, gradient_accumulation_steps=16)
trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=seq2seq_data_collator, train_dataset=dataset_samsum_pt["train"], eval_dataset=dataset_samsum_pt["validation"])


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [18]:
trainer.train() 
# score = evaluate_summaries_pegasus( dataset_samsum["test"], rouge_metric, trainer.model, tokenizer, batch_size=2, column_text="dialogue", column_summary="summary") 
# rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names) 
# pd.DataFrame(rouge_dict, index=[f"pegasus"])

The following columns in the training set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: id, dialogue, summary. If id, dialogue, summary are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
/home/users/sunhongchao/miniconda3/envs/qag_39/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14732
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 230
/home/users/sunhongchao/miniconda3/envs/qag_39/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68:

Step,Training Loss,Validation Loss
